In [1]:
# train_color_classifier.py
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

# Paths
train_dir = "data/train"
val_dir = "data/val"
save_dir = "models"
os.makedirs(save_dir, exist_ok=True)
save_path = os.path.join(save_dir, "color_classifier.h5")

# Hyperparams
IMG_SIZE = (160, 160)
BATCH = 32
EPOCHS = 12
LR = 1e-4

# Data augmentation / generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.12,
    height_shift_range=0.12,
    shear_range=0.12,
    zoom_range=0.12,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH,
    class_mode='categorical'
)

val_gen = val_datagen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH,
    class_mode='categorical'
)

# Model: MobileNetV2 base
base = MobileNetV2(input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3), include_top=False, weights='imagenet')
x = base.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
preds = Dense(train_gen.num_classes, activation='softmax')(x)
model = Model(inputs=base.input, outputs=preds)

# Freeze base for a few epochs, then optionally unfreeze
for layer in base.layers:
    layer.trainable = False

model.compile(optimizer=Adam(LR), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

callbacks = [
    ModelCheckpoint(save_path, monitor='val_accuracy', save_best_only=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)
]

model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS,
    callbacks=callbacks
)

# After initial training, optionally unfreeze some top layers and fine-tune:
for layer in base.layers[-50:]:
    layer.trainable = True

model.compile(optimizer=Adam(LR/10), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_gen, validation_data=val_gen, epochs=6, callbacks=callbacks)
print("Best model saved to:", save_path)
print("Class indices:", train_gen.class_indices)


Found 5804 images belonging to 2 classes.
Found 1460 images belonging to 2 classes.
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 6s 1us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 160, 160,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 80, 80,    │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 80, 80,    │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 80, 80,    │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 80, 80,    │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 80, 80,    │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 80, 80,    │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 80, 80,    │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 80, 80,    │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 80, 80,    │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 80, 80,    │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 80, 80,    │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 81, 81,    │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 40, 40,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 40, 40,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 40, 40,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 40, 40,    │      2,304 │ block_1_depthwis

 Total params: 2,422,210 (9.24 MB)

 Trainable params: 164,226 (641.51 KB)

...

182/182 ━━━━━━━━━━━━━━━━━━━━ 157s 863ms/step - accuracy: 0.9740 - loss: 0.0760 - val_accuracy: 0.9671 - val_loss: 0.1041 - learning_rate: 1.0000e-04
Epoch 1/6
182/182 ━━━━━━━━━━━━━━━━━━━━ 0s 512ms/step - accuracy: 0.9502 - loss: 0.1395
Epoch 1: val_accuracy improved from 0.96712 to 0.97192, saving model to models\color_classifier.h5


182/182 ━━━━━━━━━━━━━━━━━━━━ 139s 589ms/step - accuracy: 0.9503 - loss: 0.1393 - val_accuracy: 0.9719 - val_loss: 0.0804 - learning_rate: 1.0000e-05
Epoch 2/6
182/182 ━━━━━━━━━━━━━━━━━━━━ 0s 497ms/step - accuracy: 0.9695 - loss: 0.0822
Epoch 2: val_accuracy improved from 0.97192 to 0.97808, saving model to models\color_classifier.h5


182/182 ━━━━━━━━━━━━━━━━━━━━ 104s 572ms/step - accuracy: 0.9695 - loss: 0.0822 - val_accuracy: 0.9781 - val_loss: 0.0693 - learning_rate: 1.0000e-05
Epoch 3/6
182/182 ━━━━━━━━━━━━━━━━━━━━ 0s 877ms/step - accuracy: 0.9766 - loss: 0.0665
Epoch 3: val_accuracy did not improve from 0.97808
182/182 ━━━━━━━━━━━━━━━━━━━━ 177s 976ms/step - accuracy: 0.9766 - loss: 0.0665 - val_accuracy: 0.9774 - val_loss: 0.0610 - learning_rate: 1.0000e-05
Epoch 4/6
182/182 ━━━━━━━━━━━━━━━━━━━━ 0s 892ms/step - accuracy: 0.9781 - loss: 0.0715
Epoch 4: val_accuracy improved from 0.97808 to 0.97877, saving model to models\color_classifier.h5


182/182 ━━━━━━━━━━━━━━━━━━━━ 206s 997ms/step - accuracy: 0.9781 - loss: 0.0715 - val_accuracy: 0.9788 - val_loss: 0.0554 - learning_rate: 1.0000e-05
Epoch 5/6
182/182 ━━━━━━━━━━━━━━━━━━━━ 0s 891ms/step - accuracy: 0.9800 - loss: 0.0570
Epoch 5: val_accuracy improved from 0.97877 to 0.98014, saving model to models\color_classifier.h5


182/182 ━━━━━━━━━━━━━━━━━━━━ 202s 994ms/step - accuracy: 0.9800 - loss: 0.0571 - val_accuracy: 0.9801 - val_loss: 0.0537 - learning_rate: 1.0000e-05
Epoch 6/6
182/182 ━━━━━━━━━━━━━━━━━━━━ 0s 675ms/step - accuracy: 0.9790 - loss: 0.0587
Epoch 6: val_accuracy did not improve from 0.98014
182/182 ━━━━━━━━━━━━━━━━━━━━ 135s 738ms/step - accuracy: 0.9790 - loss: 0.0587 - val_accuracy: 0.9795 - val_loss: 0.0502 - learning_rate: 1.0000e-05
Best model saved to: models\color_classifier.h5
Class indices: {'blue': 0, 'other': 1}
